In [ ]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
data = []
labels = []
rows = open("../input/gtsrb-german-traffic-sign/Train.csv").read().strip().split("\n")[1:]

In [ ]:
rows[1:6]

In [ ]:
img_path="../input/gtsrb-german-traffic-sign/"


imgs=[]
for row in rows:
    path=img_path+row.split(',')[-1]
    label=row.split(',')[-2]
    d=dict()
    d["path"]=path
    d["label"]=label
    imgs.append(d)
imgs[:6]

In [ ]:
x,y = [], []
for img in imgs:
    image=cv2.imread(str(img["path"]))
    image=cv2.resize(image,(32,32))
    x.append(image)
    y.append(int(img["label"]))

In [ ]:
x = np.array(x)
y = np.array(y)
x_train = x 
y_train = y


In [ ]:
x_test,y_test = [], []
rows = open("../input/gtsrb-german-traffic-sign/Test.csv").read().strip().split("\n")[1:]

In [ ]:
img_path="../input/gtsrb-german-traffic-sign/"
imgs=[]
for row in rows:
    path=img_path+row.split(',')[-1]
    label=row.split(',')[-2]
    d=dict()
    d["path"]=path
    d["label"]=label
    imgs.append(d)


for img in imgs:
    image=cv2.imread(str(img["path"]))
    image=cv2.resize(image,(32,32))
    x_test.append(image)
    y_test.append(int(img["label"]))

In [ ]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
x_train = np.sum(x_train/3, axis=3, keepdims = True)
x_test =  np.sum(x_test/3, axis=3, keepdims = True)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,)

datagen.fit(x_train, augment=True, seed=1)



print("\ntraining_shape: ", x_train.shape,y_train.shape)
print("testing_shape: ", x_test.shape,y_test.shape)
y_train = tf.one_hot(y_train,43)
y_test = tf.one_hot(y_test,43)

In [ ]:
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout

In [ ]:
model = tf.keras.Sequential()

model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu", input_shape= (32, 32, 1)))
model.add((Conv2D(filters=32, kernel_size=(5,5), activation="relu")))

# model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(2,return_sequences=False, dropout=0.25, 
# recurrent_dropout=0.1)))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add((Conv2D(filters=64,kernel_size=(3,3),activation="relu"))) 

model.add((MaxPool2D(pool_size=(2,2))))

model.add(Dropout(rate=0.25))


model.add(Flatten()) 
model.add(Dense(256, activation="relu"))

model.add(Dropout(rate=0.40))

model.add(Dense(43, activation="softmax")) 

model.build((None, 32, 32, 1)) # `input_shape` is the shape of the input data
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
epochs = 10
history = model.fit(x_train, y_train, epochs=epochs,batch_size=64, validation_data=(x_train, y_train))

In [ ]:
score = model.evaluate(x_train, y_train)
print('Train Accuracy: {}'.format(score[1]))

score = model.evaluate(x_test, y_test)
print('Test Accuracy: {}'.format(score[1]))